In [3]:
!pip install pretty_midi pyfluidsynth openai setuptools


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from pathlib import Path
import pretty_midi
import fluidsynth
import time
import os

/Users/declanclowry/music_ai_hackathon/MAIHack_PosyVibe/venv/lib/python3.12/site-packages/pretty_midi/instrument.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
## goal - generate these programatically

### keys (note - all should repeat from 12 - 12 = 0 - it might make sense to autoconstruct the full list of notes in midi or at least 3-4 octaves for each instrument)

major_intervals = [0,2,4,5,7,9,11] # happy uplifting confident resolved
natural_minor_intervals = [0,2,3,5,7,8,10] # sad introspective moody
pentatonic_major_intervals = [0,2,4,7,9] # happy simple confident exuberant open, folksy
pentatonic_minor_intervals = [0,3,5,7,10] # dark, rocky, exciting, but sad
blues_minor_intervals = [0,3,5,6,7,10] # expressive, sorrowful, defiant
harmonic_minor_intervals = [0,2,3,5,7,8,11] # exotic dramatic mysterious tense
melodic_minor_ascending_intervals = [0,2,3,5,7,9,11] # jazzy, good to blur between major and minor
dorian_mode_intervals = [0,2,3,5,7,9,10] # cool hopeful jazzy melancholy calm
phrygian_mode_intervals = [0,1,3,5,7,8,10] # very minor unsettling tense 
lydian_mode_intervals = [0,2,4,6,7,9,11] # major, epic, film score like impressive and confident
mixolydian_mode_intervals = [0,2,4,5,7,9,10] # rocky, funky, bluesy, fun
locrian_mode_intervals = [0,1,3,5,6,8,10] # unresolved, mysterious

# japanese scales
japanese_in_scale_intervals = [0,1,5,7,8] # sparse and haunting
japanese_yo_scale_intervals = [0,2,5,7,9] # bright sounding and used in traditional folk cheerful fresh 
japanese_hirajoushi_intervals = [0,2,3,7,8] # traditional, expressive, used in koto music sombre, slow, calm
japanese_iwato_scale_intervals = [0,1,5,6,10] # mysterious and resonant edgy
japanese_kumoi_scale_intervals = [0,2,3,7,9] # evokes melancholy and sadness and nostalgia


### define chords

c_major = [[60,64,67]]

instrument_matrix = {
    "lead": {
        "program": 82,
        "notes":[
            {
                "velocity":70,
                "pitch": 60,
                "start":0.0,
                "end":2.0
            },
            {
                "velocity":80,
                "pitch": 64,
                "start":2.0,
                "end":4.0
            },
            {
                "velocity":80,
                "pitch": 64,
                "start":4.0,
                "end":5.0
            },


        ]
    }
}

In [6]:
# --- Step 1: Create a new PrettyMIDI object
midi = pretty_midi.PrettyMIDI()
# --- Step 2: Add instruments
# 🎹 Pad (channel 0)
pad = pretty_midi.Instrument(program=89)  # GM program 89 = Pad 1 (New Age)
pad.notes.append(pretty_midi.Note(velocity=80, pitch=60, start=0.0, end=2.0))
pad.notes.append(pretty_midi.Note(velocity=80, pitch=64, start=2.0, end=4.0))
pad.notes.append(pretty_midi.Note(velocity=80, pitch=67, start=4.0, end=6.0))

# 🎸 Bass (channel 1)
bass = pretty_midi.Instrument(program=33)  # Fingered Bass
bass.notes.append(pretty_midi.Note(velocity=100, pitch=43, start=0.0, end=1.5))
bass.notes.append(pretty_midi.Note(velocity=100, pitch=45, start=2.0, end=3.5))

# 🥁 Drums (channel 9)
drums = pretty_midi.Instrument(program=0, is_drum=True)
drums.notes.append(pretty_midi.Note(velocity=127, pitch=36, start=0.0, end=0.1))  # Kick
drums.notes.append(pretty_midi.Note(velocity=100, pitch=38, start=1.0, end=1.1))  # Snare
drums.notes.append(pretty_midi.Note(velocity=100, pitch=42, start=2.0, end=2.1))  # Hi-hat

# Add instruments to MIDI
midi.instruments += [pad, bass, drums]



In [7]:
# --- Step 3: Save to file
midi_path = "mood_music_tester.mid"
midi.write(midi_path)
print(f"Saved MIDI file to {midi_path}")


Saved MIDI file to mood_music_tester.mid


In [8]:

# --- Step 4: Play it back using pyfluidsynth
# Load a soundfont (make sure you have one)
soundfont_path = "resources/FatBoy-v0.786.sf2"  # Replace with the path to your .sf2 file
assert os.path.exists(soundfont_path), f"SoundFont not found: {soundfont_path}"

# Initialize FluidSynth
fs = fluidsynth.Synth()
fs.start(driver="coreaudio" if os.name == "posix" else "dsound")
sfid = fs.sfload(soundfont_path)
fs.program_select(0, sfid, 0, 89)  # Pad
fs.program_select(1, sfid, 0, 33)  # Bass
fs.program_select(9, sfid, 128, 0)  # Drums on channel 9 (bank 128)



0

In [9]:

# Playback from PrettyMIDI
for instrument in midi.instruments:
    for note in instrument.notes:
        channel = 9 if instrument.is_drum else midi.instruments.index(instrument)
        fs.noteon(channel, note.pitch, note.velocity)
        time.sleep(note.end - note.start)
        fs.noteoff(channel, note.pitch)

# Shutdown synth
fs.delete()




In [32]:
!pip install openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [33]:
!pip install python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [34]:
# Set up OpenAI connection
import openai
import dotenv
import os
dotenv.load_dotenv()


client = openai.OpenAI(api_key=os.getenv("OPENAI_KEY"))

MODEL = "gpt-4.1" ## consider 4o mini or anything else

def construct_messages(system_text):
    messages=[
        {"role": "system", "content": system_text}
    ]
    return messages

def run_text_prompt_with_history(client, msgs, input_text):
    msgs.append({"role": "user", "content": [
            {"type": "text", "text": input_text},
        ]})
    response = client.chat.completions.create(
        model = MODEL,
        messages = msgs,
        temperature=0.0,
    )
    msgs.append({"role": "assistant", "content": [
            {"type": "text", "text": response.choices[0].message.content},
        ]})
    return response.choices[0].message.content

def run_text_prompt(system_text, input_text, client):
    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_text},
        {"role": "user", "content": [
            {"type": "text", "text": input_text},
        ]}
    ],
    temperature=0.0,
    )
    return response.choices[0].message.content




In [35]:
### things to experiment with -> can we get it to write different types of thing 
# - maybe give more detailed instructions on drum beats to start with and see if we can get some different drum patterns
### TODO add tool / input around bpm that informs the correct length in seconds of one beat
test_ans = run_text_prompt(system_text="""
                           You are a musical composer who outputs melodies for a midi instrument to be fed into pretty_midi.
                           """
                           ,
                           input_text="Write a happy melody in F# Major lasting 8 bars",
                           client=client
                        )

In [36]:
### things to experiment with -> can we get it to write different types of thing 
# - maybe give more detailed instructions on drum beats to start with and see if we can get some different drum patterns
### TODO add tool / input around bpm that informs the correct length in seconds of one beat

SYSTEM_PROMPT = """
                           You are a talented musical composer who outputs melodies for a midi instrument to be fed into pretty_midi.
                           You can input notes for different instruments (defined by "program") choosing the velocity (loudness), pitch, start, and end in seconds.
                           Pitch is defined in semitones, where C3 is 60.
                           If the user asks you to create music at a certain BPM, you can use start and end seconds to stay quantised and make notes that fit into subdivisions or multiplications of one beat.
                           Try to make an interesting and thematic composition based on the users instructions.
                           Make your output in the following JSON configuration:
                           {
                                "lead": {
                                    "program": 82,
                                    "notes":[
                                        {
                                            "velocity":80,
                                            "pitch": 60,
                                            "start":0.0,
                                            "end":2.0
                                        },
                                        {
                                            "velocity":80,
                                            "pitch": 64,
                                            "start":2.0,
                                            "end":4.0
                                        },
                                        {
                                            "velocity":80,
                                            "pitch": 64,
                                            "start":4.0,
                                            "end":5.0
                                        },
                                    ]
                                }
                                "pad": {
                                    "program": 89,
                                    "notes":[
                                        {
                                            "velocity":80,
                                            "pitch": 60,
                                            "start":0.0,
                                            "end":2.0
                                        },
                                        {
                                            "velocity":80,
                                            "pitch": 64,
                                            "start":2.0,
                                            "end":4.0
                                        },
                                        {
                                            "velocity":80,
                                            "pitch": 64,
                                            "start":4.0,
                                            "end":5.0
                                        },
                                    ]
                                }
                            }
                           
                           In response to the user's request, write a musical piece  and output it in the JSON format described above.
                           """


In [37]:

SYSTEM_PROMPT = """
You are a talented musical composer who outputs melodies and drum patterns for a midi instrument to be fed into pretty_midi.
You can input notes for different instruments (defined by "program") choosing the velocity (loudness), pitch, start, and end in seconds.
Pitch is defined in semitones, where C3 is 60.

You can also generate drum patterns using the "drums" instrument. To do so:
- Use the key "drums" instead of a numbered program.
- Set "is_drum": true for the drum track.
- Use standard General MIDI drum pitches, such as:
    - 36 = Kick Drum
    - 38 = Snare
    - 42 = Closed Hi-Hat
    - 46 = Open Hi-Hat
    - 49 = Crash Cymbal
    - 51 = Ride Cymbal
    - 39 = Clap
    - 41 = Low Tom
    - 45 = Mid Tom
    - 48 = High Tom
Quantise your drum hits to start and end times that make sense rhythmically. Drums are usually short in duration (e.g., 0.1s–0.3s).

If the user asks you to create music at a certain BPM, you can use start and end seconds to stay quantised and make notes that fit into subdivisions or multiplications of one beat.

Try to make an interesting and thematic composition based on the user's instructions.

Make your output in the following JSON configuration:

{
    "lead": {
        "program": 82,
        "notes": [
            {"velocity": 80, "pitch": 60, "start": 0.0, "end": 2.0},
            {"velocity": 80, "pitch": 64, "start": 2.0, "end": 4.0}
        ]
    },
    "pad": {
        "program": 89,
        "notes": [
            {"velocity": 80, "pitch": 60, "start": 0.0, "end": 2.0},
            {"velocity": 80, "pitch": 64, "start": 2.0, "end": 4.0}
        ]
    },
    "drums": {
        "is_drum": true,
        "notes": [
            {"velocity": 127, "pitch": 36, "start": 0.0, "end": 0.1},  // Kick
            {"velocity": 100, "pitch": 38, "start": 1.0, "end": 1.1},  // Snare
            {"velocity": 100, "pitch": 42, "start": 2.0, "end": 2.1}   // Closed Hi-Hat
        ]
    }
}

In response to the user's request, write a musical piece and output it in the JSON format described above.
Your response will be fed directly into a music production software so you cannot skip and repeats or rows, it will not be read by a human so make it suitable for the machine reading the JSON dict to make music.
"""


In [38]:
test_ans = run_text_prompt(system_text=SYSTEM_PROMPT,
                           input_text="Write a happy melody in F# Major lasting 8 bars",
                           client=client
                        )

print(test_ans)

{
    "lead": {
        "program": 81,
        "notes": [
            {"velocity": 90, "pitch": 66, "start": 0.0, "end": 0.5},
            {"velocity": 90, "pitch": 68, "start": 0.5, "end": 1.0},
            {"velocity": 90, "pitch": 70, "start": 1.0, "end": 1.5},
            {"velocity": 90, "pitch": 73, "start": 1.5, "end": 2.0},
            {"velocity": 90, "pitch": 75, "start": 2.0, "end": 2.5},
            {"velocity": 90, "pitch": 73, "start": 2.5, "end": 3.0},
            {"velocity": 90, "pitch": 70, "start": 3.0, "end": 3.5},
            {"velocity": 90, "pitch": 68, "start": 3.5, "end": 4.0},
            {"velocity": 90, "pitch": 66, "start": 4.0, "end": 4.5},
            {"velocity": 90, "pitch": 68, "start": 4.5, "end": 5.0},
            {"velocity": 90, "pitch": 70, "start": 5.0, "end": 5.5},
            {"velocity": 90, "pitch": 75, "start": 5.5, "end": 6.0},
            {"velocity": 90, "pitch": 73, "start": 6.0, "end": 6.5},
            {"velocity": 90, "pitch": 70, "st

In [39]:
test_ans_2 = run_text_prompt(system_text=SYSTEM_PROMPT,
                           input_text="Write an intense melody in D Minor lasting 16 bars at 140 bpm with an interesting rhythm",
                           client=client
)

print(test_ans_2)

KeyboardInterrupt: 

In [ ]:
test_ans_3 = run_text_prompt(system_text=SYSTEM_PROMPT,
                           input_text="Write an energising drum and bass piece for 32 bars at 140 bpm in F# Minor",
                           client=client
)

print(test_ans_3)

{
    "lead": {
        "program": 81,
        "notes": [
            {"velocity": 95, "pitch": 66, "start": 0.0, "end": 0.5},
            {"velocity": 92, "pitch": 69, "start": 0.5, "end": 1.0},
            {"velocity": 90, "pitch": 71, "start": 1.0, "end": 1.5},
            {"velocity": 95, "pitch": 73, "start": 1.5, "end": 2.0},
            {"velocity": 98, "pitch": 74, "start": 2.0, "end": 2.5},
            {"velocity": 95, "pitch": 73, "start": 2.5, "end": 3.0},
            {"velocity": 92, "pitch": 71, "start": 3.0, "end": 3.5},
            {"velocity": 90, "pitch": 69, "start": 3.5, "end": 4.0},
            {"velocity": 95, "pitch": 66, "start": 4.0, "end": 4.5},
            {"velocity": 92, "pitch": 69, "start": 4.5, "end": 5.0},
            {"velocity": 90, "pitch": 71, "start": 5.0, "end": 5.5},
            {"velocity": 95, "pitch": 73, "start": 5.5, "end": 6.0},
            {"velocity": 98, "pitch": 74, "start": 6.0, "end": 6.5},
            {"velocity": 95, "pitch": 73, "st

In [ ]:
import json
import ast

def parse_and_clean_instrument_matrix(input_str):
    try:
        # Safely parse the string to a dict (no `eval`!)
        raw_matrix = ast.literal_eval(input_str)
    except (SyntaxError, ValueError) as e:
        print("❌ Error parsing input string:", e)
        return {}

    cleaned_matrix = {}

    for name, instrument in raw_matrix.items():
        if 'program' not in instrument or 'notes' not in instrument:
            continue  # skip if essential fields missing

        try:
            program = int(instrument['program'])
        except (ValueError, TypeError):
            continue

        cleaned_notes = []
        for note in instrument['notes']:
            try:
                velocity = int(note.get('velocity', 100))
                pitch = int(note.get('pitch'))
                start = float(note.get('start'))
                end = float(note.get('end'))
                if end <= start:
                    continue
                cleaned_notes.append({
                    "velocity": velocity,
                    "pitch": pitch,
                    "start": start,
                    "end": end
                })
            except (ValueError, TypeError):
                continue  # skip malformed notes

        if cleaned_notes:
            cleaned_matrix[name] = {
                "program": program,
                "notes": cleaned_notes
            }

    return cleaned_matrix

cleaned_matrix = parse_and_clean_instrument_matrix(test_ans_3)
print(cleaned_matrix)

    


❌ Error parsing input string: invalid decimal literal (<unknown>, line 120)
{}


In [10]:
cleaned_matrix = {
    "lead": {
        "program": 81,
        "notes": [
            {"velocity": 95, "pitch": 66, "start": 0.0, "end": 0.5},
            {"velocity": 92, "pitch": 69, "start": 0.5, "end": 1.0},
            {"velocity": 90, "pitch": 71, "start": 1.0, "end": 1.5},
            {"velocity": 95, "pitch": 73, "start": 1.5, "end": 2.0},
            {"velocity": 98, "pitch": 74, "start": 2.0, "end": 2.5},
            {"velocity": 95, "pitch": 73, "start": 2.5, "end": 3.0},
            {"velocity": 92, "pitch": 71, "start": 3.0, "end": 3.5},
            {"velocity": 90, "pitch": 69, "start": 3.5, "end": 4.0},
            {"velocity": 95, "pitch": 66, "start": 4.0, "end": 4.5},
            {"velocity": 92, "pitch": 69, "start": 4.5, "end": 5.0},
            {"velocity": 90, "pitch": 71, "start": 5.0, "end": 5.5},
            {"velocity": 95, "pitch": 73, "start": 5.5, "end": 6.0},
            {"velocity": 98, "pitch": 74, "start": 6.0, "end": 6.5},
            {"velocity": 95, "pitch": 73, "start": 6.5, "end": 7.0},
            {"velocity": 92, "pitch": 71, "start": 7.0, "end": 7.5},
            {"velocity": 90, "pitch": 69, "start": 7.5, "end": 8.0},
            {"velocity": 100, "pitch": 78, "start": 8.0, "end": 8.5},
            {"velocity": 98, "pitch": 76, "start": 8.5, "end": 9.0},
            {"velocity": 95, "pitch": 74, "start": 9.0, "end": 9.5},
            {"velocity": 92, "pitch": 73, "start": 9.5, "end": 10.0},
            {"velocity": 90, "pitch": 71, "start": 10.0, "end": 10.5},
            {"velocity": 95, "pitch": 69, "start": 10.5, "end": 11.0},
            {"velocity": 98, "pitch": 66, "start": 11.0, "end": 11.5},
            {"velocity": 95, "pitch": 69, "start": 11.5, "end": 12.0},
            {"velocity": 92, "pitch": 71, "start": 12.0, "end": 12.5},
            {"velocity": 90, "pitch": 73, "start": 12.5, "end": 13.0},
            {"velocity": 95, "pitch": 74, "start": 13.0, "end": 13.5},
            {"velocity": 98, "pitch": 73, "start": 13.5, "end": 14.0},
            {"velocity": 95, "pitch": 71, "start": 14.0, "end": 14.5},
            {"velocity": 92, "pitch": 69, "start": 14.5, "end": 15.0},
            {"velocity": 90, "pitch": 66, "start": 15.0, "end": 15.5},
            {"velocity": 100, "pitch": 78, "start": 15.5, "end": 16.0},
            {"velocity": 98, "pitch": 76, "start": 16.0, "end": 16.5},
            {"velocity": 95, "pitch": 74, "start": 16.5, "end": 17.0},
            {"velocity": 92, "pitch": 73, "start": 17.0, "end": 17.5},
            {"velocity": 90, "pitch": 71, "start": 17.5, "end": 18.0},
            {"velocity": 95, "pitch": 69, "start": 18.0, "end": 18.5},
            {"velocity": 98, "pitch": 66, "start": 18.5, "end": 19.0},
            {"velocity": 95, "pitch": 69, "start": 19.0, "end": 19.5},
            {"velocity": 92, "pitch": 71, "start": 19.5, "end": 20.0},
            {"velocity": 90, "pitch": 73, "start": 20.0, "end": 20.5},
            {"velocity": 95, "pitch": 74, "start": 20.5, "end": 21.0},
            {"velocity": 98, "pitch": 73, "start": 21.0, "end": 21.5},
            {"velocity": 95, "pitch": 71, "start": 21.5, "end": 22.0},
            {"velocity": 92, "pitch": 69, "start": 22.0, "end": 22.5},
            {"velocity": 90, "pitch": 66, "start": 22.5, "end": 23.0},
            {"velocity": 100, "pitch": 78, "start": 23.0, "end": 23.5},
            {"velocity": 98, "pitch": 76, "start": 23.5, "end": 24.0},
            {"velocity": 95, "pitch": 74, "start": 24.0, "end": 24.5},
            {"velocity": 92, "pitch": 73, "start": 24.5, "end": 25.0},
            {"velocity": 90, "pitch": 71, "start": 25.0, "end": 25.5},
            {"velocity": 95, "pitch": 69, "start": 25.5, "end": 26.0},
            {"velocity": 98, "pitch": 66, "start": 26.0, "end": 26.5},
            {"velocity": 95, "pitch": 69, "start": 26.5, "end": 27.0},
            {"velocity": 92, "pitch": 71, "start": 27.0, "end": 27.5},
            {"velocity": 90, "pitch": 73, "start": 27.5, "end": 28.0},
            {"velocity": 95, "pitch": 74, "start": 28.0, "end": 28.5},
            {"velocity": 98, "pitch": 73, "start": 28.5, "end": 29.0},
            {"velocity": 95, "pitch": 71, "start": 29.0, "end": 29.5},
            {"velocity": 92, "pitch": 69, "start": 29.5, "end": 30.0},
            {"velocity": 90, "pitch": 66, "start": 30.0, "end": 30.5},
            {"velocity": 100, "pitch": 78, "start": 30.5, "end": 31.0},
            {"velocity": 98, "pitch": 76, "start": 31.0, "end": 31.5},
            {"velocity": 95, "pitch": 74, "start": 31.5, "end": 32.0}
        ]
    },
    "pad": {
        "program": 89,
        "notes": [
            {"velocity": 70, "pitch": 54, "start": 0.0, "end": 8.0},
            {"velocity": 70, "pitch": 61, "start": 0.0, "end": 8.0},
            {"velocity": 70, "pitch": 66, "start": 0.0, "end": 8.0},
            {"velocity": 70, "pitch": 69, "start": 8.0, "end": 16.0},
            {"velocity": 70, "pitch": 62, "start": 8.0, "end": 16.0},
            {"velocity": 70, "pitch": 66, "start": 8.0, "end": 16.0},
            {"velocity": 70, "pitch": 71, "start": 16.0, "end": 24.0},
            {"velocity": 70, "pitch": 64, "start": 16.0, "end": 24.0},
            {"velocity": 70, "pitch": 69, "start": 16.0, "end": 24.0},
            {"velocity": 70, "pitch": 66, "start": 24.0, "end": 32.0},
            {"velocity": 70, "pitch": 61, "start": 24.0, "end": 32.0},
            {"velocity": 70, "pitch": 69, "start": 24.0, "end": 32.0}
        ]
    },
    "bass": {
        "program": 0,
        "notes": [
            {"velocity": 110, "pitch": 42, "start": 0.0, "end": 0.5},
            {"velocity": 110, "pitch": 42, "start": 1.0, "end": 1.5},
            {"velocity": 110, "pitch": 45, "start": 2.0, "end": 2.5},
            {"velocity": 110, "pitch": 47, "start": 3.0, "end": 3.5},
            {"velocity": 110, "pitch": 42, "start": 4.0, "end": 4.5},
            {"velocity": 110, "pitch": 42, "start": 5.0, "end": 5.5},
            {"velocity": 110, "pitch": 45, "start": 6.0, "end": 6.5},
            {"velocity": 110, "pitch": 47, "start": 7.0, "end": 7.5},
            {"velocity": 110, "pitch": 49, "start": 8.0, "end": 8.5},
            {"velocity": 110, "pitch": 47, "start": 9.0, "end": 9.5},
            {"velocity": 110, "pitch": 45, "start": 10.0, "end": 10.5},
            {"velocity": 110, "pitch": 42, "start": 11.0, "end": 11.5},
            {"velocity": 110, "pitch": 42, "start": 12.0, "end": 12.5},
            {"velocity": 110, "pitch": 45, "start": 13.0, "end": 13.5},
            {"velocity": 110, "pitch": 47, "start": 14.0, "end": 14.5},
            {"velocity": 110, "pitch": 42, "start": 15.0, "end": 15.5},
            {"velocity": 110, "pitch": 42, "start": 16.0, "end": 16.5},
            {"velocity": 110, "pitch": 45, "start": 17.0, "end": 17.5},
            {"velocity": 110, "pitch": 47, "start": 18.0, "end": 18.5},
            {"velocity": 110, "pitch": 42, "start": 19.0, "end": 19.5},
            {"velocity": 110, "pitch": 42, "start": 20.0, "end": 20.5},
            {"velocity": 110, "pitch": 45, "start": 21.0, "end": 21.5},
            {"velocity": 110, "pitch": 47, "start": 22.0, "end": 22.5},
            {"velocity": 110, "pitch": 49, "start": 23.0, "end": 23.5},
            {"velocity": 110, "pitch": 47, "start": 24.0, "end": 24.5},
            {"velocity": 110, "pitch": 45, "start": 25.0, "end": 25.5},
            {"velocity": 110, "pitch": 42, "start": 26.0, "end": 26.5},
            {"velocity": 110, "pitch": 42, "start": 27.0, "end": 27.5},
            {"velocity": 110, "pitch": 45, "start": 28.0, "end": 28.5},
            {"velocity": 110, "pitch": 47, "start": 29.0, "end": 29.5},
            {"velocity": 110, "pitch": 42, "start": 30.0, "end": 30.5},
            {"velocity": 110, "pitch": 42, "start": 31.0, "end": 31.5}
        ]
    },
    "drums": {
        "is_drum": True,
        "program":0,
        "notes": [
            #// Bar 1
            {"velocity": 127, "pitch": 36, "start": 0.0, "end": 0.1},
            {"velocity": 110, "pitch": 42, "start": 0.0, "end": 0.1},
            {"velocity": 110, "pitch": 42, "start": 0.25, "end": 0.35},
            {"velocity": 110, "pitch": 42, "start": 0.5, "end": 0.6},
            {"velocity": 110, "pitch": 38, "start": 0.5, "end": 0.6},
            {"velocity": 110, "pitch": 42, "start": 0.75, "end": 0.85},
            {"velocity": 127, "pitch": 36, "start": 1.0, "end": 1.1},
            {"velocity": 110, "pitch": 42, "start": 1.0, "end": 1.1},
            {"velocity": 110, "pitch": 42, "start": 1.25, "end": 1.35},
            {"velocity": 110, "pitch": 42, "start": 1.5, "end": 1.6},
            {"velocity": 110, "pitch": 38, "start": 1.5, "end": 1.6},
            {"velocity": 110, "pitch": 42, "start": 1.75, "end": 1.85},
            #// Bar 2
            {"velocity": 127, "pitch": 36, "start": 2.0, "end": 2.1},
            {"velocity": 110, "pitch": 42, "start": 2.0, "end": 2.1},
            {"velocity": 110, "pitch": 42, "start": 2.25, "end": 2.35},
            {"velocity": 110, "pitch": 42, "start": 2.5, "end": 2.6},
            {"velocity": 110, "pitch": 38, "start": 2.5, "end": 2.6},
            {"velocity": 110, "pitch": 42, "start": 2.75, "end": 2.85},
            {"velocity": 127, "pitch": 36, "start": 3.0, "end": 3.1},
            {"velocity": 110, "pitch": 42, "start": 3.0, "end": 3.1},
            {"velocity": 110, "pitch": 42, "start": 3.25, "end": 3.35},
            {"velocity": 110, "pitch": 42, "start": 3.5, "end": 3.6},
            {"velocity": 110, "pitch": 38, "start": 3.5, "end": 3.6},
            {"velocity": 110, "pitch": 42, "start": 3.75, "end": 3.85},
            #// Bar 3
            {"velocity": 127, "pitch": 36, "start": 4.0, "end": 4.1},
            {"velocity": 110, "pitch": 42, "start": 4.0, "end": 4.1},
            {"velocity": 110, "pitch": 42, "start": 4.25, "end": 4.35},
            {"velocity": 110, "pitch": 42, "start": 4.5, "end": 4.6},
            {"velocity": 110, "pitch": 38, "start": 4.5, "end": 4.6},
            {"velocity": 110, "pitch": 42, "start": 4.75, "end": 4.85},
            {"velocity": 127, "pitch": 36, "start": 5.0, "end": 5.1},
            {"velocity": 110, "pitch": 42, "start": 5.0, "end": 5.1},
            {"velocity": 110, "pitch": 42, "start": 5.25, "end": 5.35},
            {"velocity": 110, "pitch": 42, "start": 5.5, "end": 5.6},
            {"velocity": 110, "pitch": 38, "start": 5.5, "end": 5.6},
            {"velocity": 110, "pitch": 42, "start": 5.75, "end": 5.85},
            #// Bar 4
            {"velocity": 127, "pitch": 36, "start": 6.0, "end": 6.1},
            {"velocity": 110, "pitch": 42, "start": 6.0, "end": 6.1},
            {"velocity": 110, "pitch": 42, "start": 6.25, "end": 6.35},
            {"velocity": 110, "pitch": 42, "start": 6.5, "end": 6.6},
            {"velocity": 110, "pitch": 38, "start": 6.5, "end": 6.6},
            {"velocity": 110, "pitch": 42, "start": 6.75, "end": 6.85},
            {"velocity": 127, "pitch": 36, "start": 7.0, "end": 7.1},
            {"velocity": 110, "pitch": 42, "start": 7.0, "end": 7.1},
            {"velocity": 110, "pitch": 42, "start": 7.25, "end": 7.35},
            {"velocity": 110, "pitch": 42, "start": 7.5, "end": 7.6},
            {"velocity": 110, "pitch": 38, "start": 7.5, "end": 7.6},
            {"velocity": 110, "pitch": 42, "start": 7.75, "end": 7.85},
            {
                "velocity": 127,
                "pitch": 36,
                "start": 4.0,
                "end": 4.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 4.0,
                "end": 4.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 4.25,
                "end": 4.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 4.5,
                "end": 4.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 4.5,
                "end": 4.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 4.75,
                "end": 4.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 5.0,
                "end": 5.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 5.0,
                "end": 5.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 5.25,
                "end": 5.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 5.5,
                "end": 5.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 5.5,
                "end": 5.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 5.75,
                "end": 5.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 6.0,
                "end": 6.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.0,
                "end": 6.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.25,
                "end": 6.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.5,
                "end": 6.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 6.5,
                "end": 6.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.75,
                "end": 6.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 7.0,
                "end": 7.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.0,
                "end": 7.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.25,
                "end": 7.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.5,
                "end": 7.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 7.5,
                "end": 7.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.75,
                "end": 7.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 6.0,
                "end": 6.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.0,
                "end": 6.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.25,
                "end": 6.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.5,
                "end": 6.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 6.5,
                "end": 6.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 6.75,
                "end": 6.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 7.0,
                "end": 7.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.0,
                "end": 7.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.25,
                "end": 7.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.5,
                "end": 7.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 7.5,
                "end": 7.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 7.75,
                "end": 7.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 8.0,
                "end": 8.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.0,
                "end": 8.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.25,
                "end": 8.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.5,
                "end": 8.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 8.5,
                "end": 8.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.75,
                "end": 8.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 9.0,
                "end": 9.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.0,
                "end": 9.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.25,
                "end": 9.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.5,
                "end": 9.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 9.5,
                "end": 9.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.75,
                "end": 9.85
            },
            {
                "velocity": 127,
                "pitch": 49,
                "start": 8.0,
                "end": 8.2
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 8.0,
                "end": 8.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.0,
                "end": 8.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.25,
                "end": 8.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.5,
                "end": 8.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 8.5,
                "end": 8.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 8.75,
                "end": 8.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 9.0,
                "end": 9.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.0,
                "end": 9.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.25,
                "end": 9.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.5,
                "end": 9.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 9.5,
                "end": 9.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 9.75,
                "end": 9.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 10.0,
                "end": 10.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.0,
                "end": 10.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.25,
                "end": 10.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.5,
                "end": 10.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 10.5,
                "end": 10.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.75,
                "end": 10.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 11.0,
                "end": 11.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.0,
                "end": 11.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.25,
                "end": 11.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.5,
                "end": 11.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 11.5,
                "end": 11.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.75,
                "end": 11.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 10.0,
                "end": 10.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.0,
                "end": 10.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.25,
                "end": 10.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.5,
                "end": 10.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 10.5,
                "end": 10.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 10.75,
                "end": 10.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 11.0,
                "end": 11.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.0,
                "end": 11.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.25,
                "end": 11.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.5,
                "end": 11.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 11.5,
                "end": 11.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 11.75,
                "end": 11.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 12.0,
                "end": 12.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.0,
                "end": 12.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.25,
                "end": 12.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.5,
                "end": 12.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 12.5,
                "end": 12.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.75,
                "end": 12.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 13.0,
                "end": 13.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.0,
                "end": 13.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.25,
                "end": 13.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.5,
                "end": 13.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 13.5,
                "end": 13.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.75,
                "end": 13.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 12.0,
                "end": 12.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.0,
                "end": 12.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.25,
                "end": 12.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.5,
                "end": 12.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 12.5,
                "end": 12.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 12.75,
                "end": 12.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 13.0,
                "end": 13.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.0,
                "end": 13.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.25,
                "end": 13.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.5,
                "end": 13.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 13.5,
                "end": 13.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 13.75,
                "end": 13.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 14.0,
                "end": 14.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.0,
                "end": 14.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.25,
                "end": 14.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.5,
                "end": 14.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 14.5,
                "end": 14.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.75,
                "end": 14.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 15.0,
                "end": 15.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.0,
                "end": 15.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.25,
                "end": 15.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.5,
                "end": 15.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 15.5,
                "end": 15.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.75,
                "end": 15.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 14.0,
                "end": 14.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.0,
                "end": 14.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.25,
                "end": 14.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.5,
                "end": 14.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 14.5,
                "end": 14.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 14.75,
                "end": 14.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 15.0,
                "end": 15.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.0,
                "end": 15.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.25,
                "end": 15.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.5,
                "end": 15.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 15.5,
                "end": 15.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 15.75,
                "end": 15.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 16.0,
                "end": 16.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.0,
                "end": 16.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.25,
                "end": 16.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.5,
                "end": 16.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 16.5,
                "end": 16.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.75,
                "end": 16.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 17.0,
                "end": 17.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.0,
                "end": 17.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.25,
                "end": 17.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.5,
                "end": 17.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 17.5,
                "end": 17.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.75,
                "end": 17.85
            },
            {
                "velocity": 127,
                "pitch": 49,
                "start": 16.0,
                "end": 16.2
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 16.0,
                "end": 16.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.0,
                "end": 16.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.25,
                "end": 16.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.5,
                "end": 16.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 16.5,
                "end": 16.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 16.75,
                "end": 16.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 17.0,
                "end": 17.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.0,
                "end": 17.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.25,
                "end": 17.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.5,
                "end": 17.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 17.5,
                "end": 17.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 17.75,
                "end": 17.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 18.0,
                "end": 18.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.0,
                "end": 18.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.25,
                "end": 18.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.5,
                "end": 18.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 18.5,
                "end": 18.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.75,
                "end": 18.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 19.0,
                "end": 19.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.0,
                "end": 19.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.25,
                "end": 19.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.5,
                "end": 19.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 19.5,
                "end": 19.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.75,
                "end": 19.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 18.0,
                "end": 18.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.0,
                "end": 18.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.25,
                "end": 18.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.5,
                "end": 18.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 18.5,
                "end": 18.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 18.75,
                "end": 18.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 19.0,
                "end": 19.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.0,
                "end": 19.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.25,
                "end": 19.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.5,
                "end": 19.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 19.5,
                "end": 19.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 19.75,
                "end": 19.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 20.0,
                "end": 20.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.0,
                "end": 20.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.25,
                "end": 20.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.5,
                "end": 20.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 20.5,
                "end": 20.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.75,
                "end": 20.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 21.0,
                "end": 21.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.0,
                "end": 21.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.25,
                "end": 21.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.5,
                "end": 21.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 21.5,
                "end": 21.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.75,
                "end": 21.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 20.0,
                "end": 20.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.0,
                "end": 20.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.25,
                "end": 20.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.5,
                "end": 20.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 20.5,
                "end": 20.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 20.75,
                "end": 20.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 21.0,
                "end": 21.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.0,
                "end": 21.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.25,
                "end": 21.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.5,
                "end": 21.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 21.5,
                "end": 21.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 21.75,
                "end": 21.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 22.0,
                "end": 22.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.0,
                "end": 22.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.25,
                "end": 22.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.5,
                "end": 22.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 22.5,
                "end": 22.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.75,
                "end": 22.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 23.0,
                "end": 23.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.0,
                "end": 23.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.25,
                "end": 23.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.5,
                "end": 23.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 23.5,
                "end": 23.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.75,
                "end": 23.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 22.0,
                "end": 22.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.0,
                "end": 22.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.25,
                "end": 22.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.5,
                "end": 22.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 22.5,
                "end": 22.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 22.75,
                "end": 22.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 23.0,
                "end": 23.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.0,
                "end": 23.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.25,
                "end": 23.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.5,
                "end": 23.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 23.5,
                "end": 23.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 23.75,
                "end": 23.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 24.0,
                "end": 24.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.0,
                "end": 24.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.25,
                "end": 24.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.5,
                "end": 24.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 24.5,
                "end": 24.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.75,
                "end": 24.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 25.0,
                "end": 25.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.0,
                "end": 25.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.25,
                "end": 25.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.5,
                "end": 25.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 25.5,
                "end": 25.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.75,
                "end": 25.85
            },
            {
                "velocity": 127,
                "pitch": 49,
                "start": 24.0,
                "end": 24.2
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 24.0,
                "end": 24.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.0,
                "end": 24.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.25,
                "end": 24.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.5,
                "end": 24.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 24.5,
                "end": 24.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 24.75,
                "end": 24.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 25.0,
                "end": 25.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.0,
                "end": 25.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.25,
                "end": 25.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.5,
                "end": 25.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 25.5,
                "end": 25.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 25.75,
                "end": 25.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 26.0,
                "end": 26.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.0,
                "end": 26.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.25,
                "end": 26.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.5,
                "end": 26.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 26.5,
                "end": 26.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.75,
                "end": 26.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 27.0,
                "end": 27.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.0,
                "end": 27.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.25,
                "end": 27.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.5,
                "end": 27.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 27.5,
                "end": 27.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.75,
                "end": 27.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 26.0,
                "end": 26.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.0,
                "end": 26.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.25,
                "end": 26.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.5,
                "end": 26.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 26.5,
                "end": 26.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 26.75,
                "end": 26.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 27.0,
                "end": 27.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.0,
                "end": 27.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.25,
                "end": 27.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.5,
                "end": 27.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 27.5,
                "end": 27.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 27.75,
                "end": 27.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 28.0,
                "end": 28.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.0,
                "end": 28.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.25,
                "end": 28.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.5,
                "end": 28.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 28.5,
                "end": 28.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.75,
                "end": 28.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 29.0,
                "end": 29.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.0,
                "end": 29.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.25,
                "end": 29.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.5,
                "end": 29.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 29.5,
                "end": 29.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.75,
                "end": 29.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 28.0,
                "end": 28.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.0,
                "end": 28.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.25,
                "end": 28.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.5,
                "end": 28.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 28.5,
                "end": 28.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 28.75,
                "end": 28.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 29.0,
                "end": 29.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.0,
                "end": 29.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.25,
                "end": 29.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.5,
                "end": 29.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 29.5,
                "end": 29.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 29.75,
                "end": 29.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 30.0,
                "end": 30.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.0,
                "end": 30.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.25,
                "end": 30.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.5,
                "end": 30.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 30.5,
                "end": 30.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.75,
                "end": 30.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 31.0,
                "end": 31.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.0,
                "end": 31.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.25,
                "end": 31.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.5,
                "end": 31.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 31.5,
                "end": 31.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.75,
                "end": 31.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 30.0,
                "end": 30.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.0,
                "end": 30.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.25,
                "end": 30.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.5,
                "end": 30.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 30.5,
                "end": 30.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 30.75,
                "end": 30.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 31.0,
                "end": 31.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.0,
                "end": 31.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.25,
                "end": 31.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.5,
                "end": 31.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 31.5,
                "end": 31.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 31.75,
                "end": 31.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 32.0,
                "end": 32.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 32.0,
                "end": 32.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 32.25,
                "end": 32.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 32.5,
                "end": 32.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 32.5,
                "end": 32.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 32.75,
                "end": 32.85
            },
            {
                "velocity": 127,
                "pitch": 36,
                "start": 33.0,
                "end": 33.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 33.0,
                "end": 33.1
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 33.25,
                "end": 33.35
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 33.5,
                "end": 33.6
            },
            {
                "velocity": 110,
                "pitch": 38,
                "start": 33.5,
                "end": 33.6
            },
            {
                "velocity": 110,
                "pitch": 42,
                "start": 33.75,
                "end": 33.85
            }

        ]
    }
}

In [15]:
def generate_instrument_from_dict(input_dict, instrument_name):
    program = input_dict[instrument_name]["program"]
    notes = input_dict[instrument_name]["notes"]
    drums = instrument_name.lower() == "drums"  # ✅ Correct drum detection

    instrument = pretty_midi.Instrument(program=program, is_drum=drums)
    for single_note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=single_note["velocity"],
                pitch=single_note["pitch"],
                start=single_note["start"],
                end=single_note["end"]
            )
        )
    return instrument

    

lead = generate_instrument_from_dict(cleaned_matrix, "lead")
pad =generate_instrument_from_dict(cleaned_matrix, "pad")
drums = generate_instrument_from_dict(cleaned_matrix, "drums")


In [16]:
# --- Step 3: Save to file
midi = pretty_midi.PrettyMIDI()
midi.instruments+=[lead,pad,drums]
midi_path = "ai_gen_score_test.mid"
midi.write(midi_path)
print(f"Saved MIDI file to {midi_path}")

Saved MIDI file to ai_gen_score_test.mid


In [14]:
fs.delete()


In [17]:
import time
import pretty_midi
import fluidsynth
import os

# --- Load SoundFont
soundfont_path = "resources/FatBoy-v0.786.sf2"
assert os.path.exists(soundfont_path), "SoundFont not found!"

fs = fluidsynth.Synth()
fs.start(driver="coreaudio" if os.name == "posix" else "dsound")
sfid = fs.sfload(soundfont_path)

# --- Channel allocation
channel_map = {}
channel_counter = 0
events = []

# --- Load MIDI file
midi = pretty_midi.PrettyMIDI("ai_gen_score_test.mid")

# --- Assign instruments to channels & collect events
for instrument in midi.instruments:
    if instrument.is_drum:
        channel = 9
        fs.program_select(channel, sfid, 128, 0)  # FatBoy drum bank
    else:
        if instrument.program not in channel_map:
            fs.program_select(channel_counter, sfid, 0, instrument.program)
            channel_map[instrument.program] = channel_counter
            channel_counter += 1
        channel = channel_map[instrument.program]

    for note in instrument.notes:
        events.append(("on", note.start, channel, note.pitch, note.velocity))
        events.append(("off", note.end, channel, note.pitch, 0))

# --- Sort events by time
events.sort(key=lambda x: x[1])

# --- Play notes in real time
start_time = time.time()
for event_type, event_time, channel, pitch, velocity in events:
    now = time.time() - start_time
    sleep_time = event_time - now
    if sleep_time > 0:
        time.sleep(sleep_time)

    if event_type == "on":
        fs.noteon(channel, pitch, velocity)
    else:
        fs.noteoff(channel, pitch)

# --- Shutdown
fs.delete()


In [19]:
import pretty_midi
import subprocess
import os

# --- Paths
midi_path = "ai_gen_score_test.mid"
soundfont_path = "resources/FatBoy-v0.786.sf2"
output_wav_path = "output.wav"

# --- Save PrettyMIDI to file (if not already saved)
midi = pretty_midi.PrettyMIDI(midi_path)  # or build your MIDI and do midi.write()
midi.write(midi_path)

# --- Render to WAV using FluidSynth CLI
subprocess.run([
    "fluidsynth",
    "-ni",
    soundfont_path,
    midi_path,
    "-F", output_wav_path,
    "-r", "44100"
], check=True)

print(f"✅ Rendered to {output_wav_path}")



FluidSynth runtime version 2.4.6
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'output.wav'..
✅ Rendered to output.wav


In [ ]:
import time
import pretty_midi
import fluidsynth
import os

# Load MIDI from PrettyMIDI object
midi = pretty_midi.PrettyMIDI("ai_gen_score_test.mid")  # or however you have it

# SoundFont
soundfont_path = "resources/FatBoy-v0.786.sf2"
assert os.path.exists(soundfont_path), "Missing soundfont!"

# Initialize FluidSynth
fs = fluidsynth.Synth()
fs.start(driver="coreaudio" if os.name == "posix" else "dsound")
sfid = fs.sfload(soundfont_path)
fs.program_select(0, sfid, 0, 81)  # lead
fs.program_select(1, sfid, 0, 89)  # synth
fs.program_select(9, sfid, 0, 0)  # Just use program 0 on channel 9

# Channel allocation (you can expand this if needed)
channel_map = {}
channel_counter = 0

# Collect note events
events = []

for instrument in midi.instruments:
    if instrument.is_drum:
        channel = 9
        fs.program_select(channel, sfid, 128, 0)
    else:
        if instrument.program not in channel_map:
            fs.program_select(channel_counter, sfid, 0, instrument.program)
            channel_map[instrument.program] = channel_counter
            channel_counter += 1
        channel = channel_map[instrument.program]

    for note in instrument.notes:
        events.append(("on", note.start, channel, note.pitch, note.velocity))
        events.append(("off", note.end, channel, note.pitch, 0))

# Sort events by time
events.sort(key=lambda x: x[1])

# Playback with scheduling
start_time = time.time()
for event_type, event_time, channel, pitch, velocity in events:
    now = time.time() - start_time
    sleep_time = event_time - now
    if sleep_time > 0:
        time.sleep(sleep_time)

    if event_type == "on":
        fs.noteon(channel, pitch, velocity)
    else:
        fs.noteoff(channel, pitch)

# Shutdown
fs.delete()


In [ ]:
import time
import fluidsynth
import os

soundfont_path = "resources/FatBoy-v0.786.sf2"
assert os.path.exists(soundfont_path), "SoundFont not found!"

fs = fluidsynth.Synth()
fs.start(driver="coreaudio" if os.name == "posix" else "dsound")

sfid = fs.sfload(soundfont_path)
fs.program_select(0, sfid, 0, 0)  # Acoustic Grand Piano on channel 0

# Play middle C (MIDI pitch 60)
fs.noteon(0, 60, 120)
time.sleep(1)
fs.noteoff(0, 60)

fs.delete()





In [1]:
import time
import fluidsynth
import os

soundfont_path = "resources/FatBoy-v0.786.sf2"
assert os.path.exists(soundfont_path), "SoundFont not found!"

fs = fluidsynth.Synth()
fs.start(driver="coreaudio" if os.name == "posix" else "dsound")

sfid = fs.sfload(soundfont_path)

# ✅ WORKING drum configuration — critical to avoid setting program manually!
fs.program_select(9, sfid, 0, 0)  # still use program 0, GM mode

# Test kick and snare
fs.noteon(10, 36, 120)  # Kick
time.sleep(0.4)
fs.noteoff(10, 36)
fs.noteon(10, 38, 120)  # Snare
time.sleep(0.4)
fs.noteoff(10, 38)

fs.delete()


In [ ]:
#### Let's put this all together


# Re-run after environment reset

import os
import openai
import dotenv
import pretty_midi
import subprocess
import ast
import json
import datetime

dotenv.load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_KEY"))

MODEL = "gpt-4.1"
SOUNDFONT_PATH = "resources/FatBoy-v0.786.sf2"
OUTPUT_MIDI_PATH = "ai_gen_score_test.mid"
OUTPUT_WAV_PATH = f"output{datetime.datetime.now()}.wav"

SYSTEM_PROMPT = """
You are a talented musical composer who outputs melodies and drum patterns for a midi instrument to be fed into pretty_midi.
You can input notes for different instruments (defined by "program") choosing the velocity (loudness), pitch, start, and end in seconds.
Pitch is defined in semitones, where C3 is 60.

You can also generate drum patterns using the "drums" instrument. To do so:
- Use the key "drums" instead of a numbered program.
- Set "is_drum": true for the drum track.
- Use standard General MIDI drum pitches, such as:
    - 36 = Kick Drum
    - 38 = Snare
    - 42 = Closed Hi-Hat
    - 46 = Open Hi-Hat
    - 49 = Crash Cymbal
    - 51 = Ride Cymbal
    - 39 = Clap
    - 41 = Low Tom
    - 45 = Mid Tom
    - 48 = High Tom
Quantise your drum hits to start and end times that make sense rhythmically. Drums are usually short in duration (e.g., 0.1s–0.3s).

If the user asks you to create music at a certain BPM, you can use start and end seconds to stay quantised and make notes that fit into subdivisions or multiplications of one beat.

Try to make an interesting and thematic composition based on the user's instructions.

Make your output in the following JSON configuration:

{
    "lead": {
        "program": 82,
        "notes": [
            {"velocity": 80, "pitch": 60, "start": 0.0, "end": 2.0},
            {"velocity": 80, "pitch": 64, "start": 2.0, "end": 4.0}
        ]
    },
    "pad": {
        "program": 89,
        "notes": [
            {"velocity": 80, "pitch": 60, "start": 0.0, "end": 2.0},
            {"velocity": 80, "pitch": 64, "start": 2.0, "end": 4.0}
        ]
    },
    "drums": {
        "is_drum": true,
        "program":0,
        "notes": [
            {"velocity": 127, "pitch": 36, "start": 0.0, "end": 0.1},
            {"velocity": 100, "pitch": 38, "start": 1.0, "end": 1.1},
            {"velocity": 100, "pitch": 42, "start": 2.0, "end": 2.1}
        ]
    }
}
Your response will be fed directly into a music production software so do not include any explanation text.
"""

SYSTEM_PROMPT = """
You are a creative, professional musical composer for AI-driven music production.
You output imaginative, realistic, and musically interesting MIDI arrangements in JSON format, which will be directly imported into a music creation tool using pretty_midi.

**Instructions:**

- Each part (e.g., "lead", "pad", "bass", "drums") is a dictionary entry.
- Each instrument entry includes:
    - "program": MIDI program number (see General MIDI for options),
    - "notes": list of notes, each with:
        - "velocity": 1–127 (how loud/strong),
        - "pitch": MIDI note number (C3 = 60),
        - "start": when (in seconds) the note begins,
        - "end": when (in seconds) the note ends (must be > start).
- Drums are a special case: use key "drums", set `"is_drum": true`, and use General MIDI drum pitches:
    - 36 = Kick Drum, 38 = Snare, 42 = Closed Hi-Hat, 46 = Open Hi-Hat,
    - 49 = Crash Cymbal, 51 = Ride, 39 = Clap, 41 = Low Tom, 45 = Mid Tom, 48 = High Tom, etc.
    - Drum notes are typically short (0.05–0.3s) and can be rapid and layered.
- **DO NOT** output anything except the JSON structure.

**Write music that is:**
- Varied in note lengths and rhythms (not just steady 8ths/quarters).
- Musically plausible and stylistically appropriate for the requested genre/mood.
- Featuring syncopation, rests, fills, and dynamic contrast where appropriate.
- With creative chord/melody/rhythm relationships (not just stepwise movement).
- For longer pieces, add development, breakdowns, fills, or section changes.
- Optionally include "bass" or "arp" instruments for richer arrangement.

### EXAMPLES

Example 1: Upbeat dance-pop groove (syncopation, variation, layered drums)
{
    "lead": {
        "program": 81,
        "notes": [
            {"velocity": 90, "pitch": 72, "start": 0.0, "end": 0.3},
            {"velocity": 88, "pitch": 74, "start": 0.4, "end": 0.8},
            {"velocity": 95, "pitch": 76, "start": 1.0, "end": 1.2},
            {"velocity": 75, "pitch": 79, "start": 1.5, "end": 2.1},
            {"velocity": 82, "pitch": 77, "start": 2.3, "end": 2.7}
        ]
    },
    "pad": {
        "program": 89,
        "notes": [
            {"velocity": 60, "pitch": 60, "start": 0.0, "end": 2.5},
            {"velocity": 65, "pitch": 67, "start": 2.5, "end": 4.0}
        ]
    },
    "bass": {
        "program": 38,
        "notes": [
            {"velocity": 100, "pitch": 43, "start": 0.0, "end": 0.4},
            {"velocity": 105, "pitch": 47, "start": 0.8, "end": 1.1},
            {"velocity": 110, "pitch": 45, "start": 1.5, "end": 1.9}
        ]
    },
    "drums": {
        "is_drum": true,
        "program": 0,
        "notes": [
            {"velocity": 120, "pitch": 36, "start": 0.0, "end": 0.09},   // Kick
            {"velocity": 85, "pitch": 42, "start": 0.25, "end": 0.33},  // Closed HH
            {"velocity": 110, "pitch": 38, "start": 0.5, "end": 0.6},   // Snare
            {"velocity": 100, "pitch": 36, "start": 1.0, "end": 1.08},  // Kick
            {"velocity": 90, "pitch": 42, "start": 1.25, "end": 1.33},  // Closed HH
            {"velocity": 120, "pitch": 49, "start": 2.0, "end": 2.15},  // Crash
            {"velocity": 110, "pitch": 38, "start": 2.5, "end": 2.6}    // Snare
        ]
    }
}

Example 2: Downtempo, moody, syncopated (rests, off-beat drums, varying lengths)
{
    "lead": {
        "program": 83,
        "notes": [
            {"velocity": 70, "pitch": 68, "start": 0.0, "end": 0.2},
            {"velocity": 75, "pitch": 71, "start": 0.45, "end": 1.0},
            {"velocity": 90, "pitch": 73, "start": 1.8, "end": 2.1},
            {"velocity": 60, "pitch": 74, "start": 2.7, "end": 3.2}
        ]
    },
    "pad": {
        "program": 92,
        "notes": [
            {"velocity": 55, "pitch": 61, "start": 0.0, "end": 2.0},
            {"velocity": 60, "pitch": 69, "start": 2.0, "end": 4.0}
        ]
    },
    "bass": {
        "program": 34,
        "notes": [
            {"velocity": 95, "pitch": 41, "start": 0.0, "end": 0.7},
            {"velocity": 80, "pitch": 43, "start": 1.1, "end": 1.7},
            {"velocity": 105, "pitch": 46, "start": 2.2, "end": 2.9}
        ]
    },
    "drums": {
        "is_drum": true,
        "program": 0,
        "notes": [
            {"velocity": 115, "pitch": 36, "start": 0.0, "end": 0.09},
            {"velocity": 85, "pitch": 42, "start": 0.18, "end": 0.22},
            {"velocity": 90, "pitch": 38, "start": 1.1, "end": 1.2},
            {"velocity": 90, "pitch": 46, "start": 1.7, "end": 1.9},
            {"velocity": 120, "pitch": 49, "start": 2.0, "end": 2.18},
            {"velocity": 85, "pitch": 42, "start": 2.5, "end": 2.58}
        ]
    }
}

Example 3: Calm, ambient, slow build (sustained notes, soft drums)
{
    "lead": {
        "program": 85,
        "notes": [
            {"velocity": 50, "pitch": 64, "start": 0.0, "end": 2.0},
            {"velocity": 52, "pitch": 67, "start": 2.2, "end": 5.0}
        ]
    },
    "pad": {
        "program": 91,
        "notes": [
            {"velocity": 40, "pitch": 60, "start": 0.0, "end": 5.0}
        ]
    },
    "drums": {
        "is_drum": true,
        "program": 0,
        "notes": [
            {"velocity": 60, "pitch": 51, "start": 1.0, "end": 1.15},  // Ride
            {"velocity": 55, "pitch": 42, "start": 3.0, "end": 3.1}   // HH
        ]
    }
}

Example 4: Fast, energetic, rhythmic (fills, offbeat drums, rapid notes)
{
    "lead": {
        "program": 82,
        "notes": [
            {"velocity": 110, "pitch": 72, "start": 0.0, "end": 0.15},
            {"velocity": 105, "pitch": 76, "start": 0.18, "end": 0.33},
            {"velocity": 120, "pitch": 79, "start": 0.4, "end": 0.55},
            {"velocity": 90, "pitch": 74, "start": 0.65, "end": 0.77},
            {"velocity": 95, "pitch": 81, "start": 0.88, "end": 1.0}
        ]
    },
    "pad": {
        "program": 93,
        "notes": [
            {"velocity": 75, "pitch": 66, "start": 0.0, "end": 2.0},
            {"velocity": 80, "pitch": 69, "start": 2.1, "end": 4.0}
        ]
    },
    "bass": {
        "program": 39,
        "notes": [
            {"velocity": 127, "pitch": 36, "start": 0.0, "end": 0.18},
            {"velocity": 110, "pitch": 39, "start": 0.19, "end": 0.33},
            {"velocity": 127, "pitch": 36, "start": 0.34, "end": 0.48},
            {"velocity": 110, "pitch": 41, "start": 0.49, "end": 0.60}
        ]
    },
    "drums": {
        "is_drum": true,
        "program": 0,
        "notes": [
            {"velocity": 127, "pitch": 36, "start": 0.0, "end": 0.09},
            {"velocity": 100, "pitch": 42, "start": 0.12, "end": 0.16},
            {"velocity": 127, "pitch": 38, "start": 0.2, "end": 0.3},
            {"velocity": 80, "pitch": 39, "start": 0.34, "end": 0.38},
            {"velocity": 100, "pitch": 42, "start": 0.43, "end": 0.47},
            {"velocity": 127, "pitch": 36, "start": 0.5, "end": 0.58},
            {"velocity": 127, "pitch": 49, "start": 1.0, "end": 1.18}
        ]
    }
}

**Never just output repeated or scale patterns. Use musical variety, syncopation, creative rhythms, fills, and rests. The output should be lively, human-like, and suitable for modern electronic/rock/pop/ambient music as appropriate to the user's instructions.**
Your response will be fed directly into a music production software so do not include any explanation text.

---
Reference: Common Scales, Modes, and Their Moods

Use the following interval patterns (semitones from tonic) when composing in the specified keys/modes:

- **Major**: [0,2,4,5,7,9,11] — happy, uplifting, confident, resolved
- **Natural Minor**: [0,2,3,5,7,8,10] — sad, introspective, moody
- **Pentatonic Major**: [0,2,4,7,9] — happy, simple, open, folksy
- **Pentatonic Minor**: [0,3,5,7,10] — dark, rocky, exciting, but sad
- **Blues Minor**: [0,3,5,6,7,10] — expressive, sorrowful, defiant
- **Harmonic Minor**: [0,2,3,5,7,8,11] — exotic, dramatic, mysterious, tense
- **Melodic Minor Ascending**: [0,2,3,5,7,9,11] — jazzy, ambiguous
- **Dorian Mode**: [0,2,3,5,7,9,10] — cool, jazzy, melancholy, calm
- **Phrygian Mode**: [0,1,3,5,7,8,10] — very minor, unsettling, tense
- **Lydian Mode**: [0,2,4,6,7,9,11] — epic, film score, impressive
- **Mixolydian Mode**: [0,2,4,5,7,9,10] — rocky, funky, fun
- **Locrian Mode**: [0,1,3,5,6,8,10] — unresolved, mysterious

**Japanese Scales:**
- **In Scale**: [0,1,5,7,8] — sparse, haunting
- **Yo Scale**: [0,2,5,7,9] — bright, traditional, cheerful
- **Hirajoushi**: [0,2,3,7,8] — koto music, sombre, calm
- **Iwato**: [0,1,5,6,10] — mysterious, edgy
- **Kumoi**: [0,2,3,7,9] — melancholy, nostalgic

*When a specific style, mood, or scale is requested, use these intervals as the basis for melody and chord selection.*

### Reference: Chord Notation, Intervals, and MIDI Note Arrays

When you see a chord progression like I–V–vi–IV, use the following rules to convert chord names into MIDI notes:

- Each chord is built on a **root note**. For example, in the key of C, “I” is C major (root = C, MIDI 60).
- Use the **intervals below** to construct each chord:
    - **Major chord**: [0, 4, 7] (root, major third, perfect fifth)
    - **Minor chord**: [0, 3, 7] (root, minor third, perfect fifth)
    - **Dominant 7th chord**: [0, 4, 7, 10]
    - **Minor 7th chord**: [0, 3, 7, 10]
    - **Major 7th chord**: [0, 4, 7, 11]
- Add each interval (in semitones) to the root note’s MIDI number to get the notes of the chord.

**Examples in the key of C (C = 60):**
- **C (I)**: C major = MIDI [60, 64, 67]
- **G (V)**: G major = MIDI [67, 71, 74]
- **Am (vi)**: A minor = MIDI [69, 72, 76]
- **F (IV)**: F major = MIDI [65, 69, 72]
- **C7**: C dominant seventh = MIDI [60, 64, 67, 70]
- **Dm7**: D minor seventh = MIDI [62, 65, 69, 72]

**How to represent a chord progression in JSON for pretty_midi:**
- For each chord, add all chord notes with the same start and end time (to make a block chord).
- Change chords at regular intervals (for example, each bar is 1.0 second).

**Example: I–V–vi–IV in C (each chord is 1 bar/second):**
```json
{
  "pad": {
    "program": 89,
    "notes": [
      {"velocity": 90, "pitch": 60, "start": 0.0, "end": 1.0},
      {"velocity": 90, "pitch": 64, "start": 0.0, "end": 1.0},
      {"velocity": 90, "pitch": 67, "start": 0.0, "end": 1.0},

      {"velocity": 90, "pitch": 67, "start": 1.0, "end": 2.0},
      {"velocity": 90, "pitch": 71, "start": 1.0, "end": 2.0},
      {"velocity": 90, "pitch": 74, "start": 1.0, "end": 2.0},

      {"velocity": 90, "pitch": 69, "start": 2.0, "end": 3.0},
      {"velocity": 90, "pitch": 72, "start": 2.0, "end": 3.0},
      {"velocity": 90, "pitch": 76, "start": 2.0, "end": 3.0},

      {"velocity": 90, "pitch": 65, "start": 3.0, "end": 4.0},
      {"velocity": 90, "pitch": 69, "start": 3.0, "end": 4.0},
      {"velocity": 90, "pitch": 72, "start": 3.0, "end": 4.0}
    ]
  }
}



Sample chord progressions for different moods (use as inspiration - you can use others too don't be limited)

- Pop major: I–V–vi–IV (e.g., C–G–Am–F in C major)
- Minor ballad: i–VI–III–VII (e.g., Am–F–C–G in A minor)
- Jazz dorian: i7–IV7 (e.g., Dm7–G7 in D dorian)
- Japanese in: i–bII–V (e.g., C–Db–G for C in scale)
- Blues: I7–IV7–V7 (e.g., A7–D7–E7 in A blues)
1. Hopeful, triumphant pop
Mood: Inspiring, victorious
Progression: I – V – vi – I⁷ – IV
Example in C: C – G – Am – C7 – F

2. Soulful, neo-soul/R&B
Mood: Smooth, jazzy, sophisticated
Progression: ii⁷ – V⁷ – Imaj7 – vi⁷
Example in Bb: Cm7 – F7 – Bbmaj7 – Gm7

3. Classic cinematic/film score
Mood: Grand, sweeping, emotional
Progression: I – vi – IV – V – I
Example in G: G – Em – C – D – G

4. Mysterious, modal folk (Dorian)
Mood: Enigmatic, ancient, adventurous
Progression: i – VII – IV – i
Example in D Dorian: Dm – C – G – Dm

5. Modern indie/alt rock
Mood: Bittersweet, reflective
Progression: vi – IV – I – V
Example in A minor/C major: Am – F – C – G

"""


SYSTEM_PROMPT = """
You are a creative, professional musical composer.  
Your job: **deliver expressive, varied, human-sounding MIDI arrangements** in pure JSON (no comments, no prose) for pretty_midi.

────────────────  CORE FORMAT  ────────────────
Each top-level key (e.g. "lead", "pad", "bass", "drums", "arp") is an instrument.control_changes.
Add a top-level key "bpm" with the piece’s tempo in beats-per-minute. Use the bpm to correctly set the start and end time in seconds for each note.

{
  "bpm": 80,
  "lead":  { "program": 84, "notes": [ … ] },
  "pad":   { "program": 89, "notes": [ … ] },
  "bass":  { "program": 34, "notes": [ … ] },
  "drums": { "is_drum": true, "program": 0, "notes": [ … ] }
}

Every note object =  
{ "velocity": 1-127, "pitch": MIDI#, "start": sec, "end": sec  (> start) }

• **Drums**: key = "drums", set `"is_drum": true`, use GM pitches  
  36 Kick  38 Snare  42 CHH  46 OHH  49 Crash  51 Ride  etc.  
  Hits 0.05–0.3 s; include ghost notes, fills, and bar-4 / bar-8 variations.

────────────────  COMPOSITION RULES  ────────────────
1. **Follow the requested key/scale & chord progression**  
   → Build chords from the interval table below.  
   → Bass **begins each chord/bar on its root** (passing tones & octaves welcome).

2. **Rhythmic & dynamic variety**  
   • Mix at least three different note lengths per part.  
   • Use syncopation, off-beats, rests, and accented/ghost velocities.

3. **Melody / Lead Phrase & motif**  
   • Lead should develop motifs (repeat & vary), not just climb scales and have varied note lengths in cool catchy melodies.
   • Use long sustained notes, only syncopate occasionally.
   • Don't make velocity too loud, it can be just 1 or 2 above pad.
   • Pads may use inversions, broken chords, or stabs—not only block chords.  
   • If you include any arps they should alternate directions or rhythms, not a constant stair-step, and low velocity.
   • Depending on the mood and genre, you can use some long notes up to or over a whole bar for melodies.

4. **Drum groove**  
   • Kick + snare pocket; hats/perc drive; fill or crash every 4–8 bars.  
   • This is primarily electronic music so even for calm tracks keep a steady beat ang kick drums. Use high velocity for kicks.

5. **Ending**  
   The composition will be looped, so you MUST end all parts together at the end of the last bar to avoid trailing silence and allow a smooth loop at the end.

6. **No “repeat” instructions** – spell out every note bar-by-bar.

────────────────  REFERENCE: SCALES & MOODS  ────────────────
Major  [0,2,4,5,7,9,11]  happy | Natural Minor  [0,2,3,5,7,8,10]  sad | …  
Pentatonic Major, Blues Minor, Dorian, Lydian, Mixolydian, etc.  
Japanese: In [0,1,5,7,8], Yo [0,2,5,7,9], etc.

────────────────  REFERENCE: CHORD BUILDING  ────────────────
Intervals from root → note list:  
Major [0,4,7] | Minor [0,3,7] | Maj7 [0,4,7,11] | Dom7 [0,4,7,10] | Min7 [0,3,7,10]

Example in C:  
C  → [60,64,67] G  → [67,71,74] Am → [69,72,76] F → [65,69,72]

────────────────  MINI-EXAMPLE: 1-bar I–V–vi–IV pad  ────────────────
{ "pad": { "program":89, "notes":[
  {"velocity":80,"pitch":60,"start":0.0,"end":1.0},
  {"velocity":80,"pitch":64,"start":0.0,"end":1.0},
  {"velocity":80,"pitch":67,"start":0.0,"end":1.0},

  {"velocity":80,"pitch":67,"start":1.0,"end":2.0},
  {"velocity":80,"pitch":71,"start":1.0,"end":2.0},
  {"velocity":80,"pitch":74,"start":1.0,"end":2.0},
  … repeat for Am and F …
] } }



────────────────  SAMPLE PROGRESSIONS & MOODS  ────────────────
• Pop (hopeful):  I–V–vi–IV → C–G–Am–F  
• Film-score (grand):  I–vi–IV–V → G–Em–C–D–G  
• Neo-soul (smooth):  ii7–V7–Imaj7–vi7 → Cm7–F7–Bbmaj7–Gm7  
• Dorian folk (mysterious):  i–VII–IV–i → Dm–C–G–Dm  
• Indie (bittersweet):  vi–IV–I–V → Am–F–C–G

────────────────  EXTENDED SCALE / MOOD TABLE  ────────────────
Major                   [0,2,4,5,7,9,11]   happy, uplifting, confident
Natural Minor           [0,2,3,5,7,8,10]   sad, introspective, moody
Pentatonic Major        [0,2,4,7,9]        simple, open, folksy
Pentatonic Minor        [0,3,5,7,10]       dark, rocky, exciting
Blues Minor             [0,3,5,6,7,10]     expressive, defiant
Harmonic Minor          [0,2,3,5,7,8,11]   exotic, dramatic
Melodic Minor Ascend.   [0,2,3,5,7,9,11]   jazzy, ambiguous
Dorian                  [0,2,3,5,7,9,10]   cool, hopeful, mellow
Phrygian                [0,1,3,5,7,8,10]   tense, mysterious
Lydian                  [0,2,4,6,7,9,11]   epic, filmic, soaring
Mixolydian              [0,2,4,5,7,9,10]   blues-rock, funky
Locrian                 [0,1,3,5,6,8,10]   unresolved, eerie

Japanese In             [0,1,5,7,8]        sparse, haunting
Japanese Yo             [0,2,5,7,9]        bright, cheerful, folk
Hirajoushi              [0,2,3,7,8]        sombre, calm
Iwato                   [0,1,5,6,10]       edgy, resonant
Kumoi                   [0,2,3,7,9]        nostalgic, wistful

────────────────  TARGETED MOOD PLAYBOOKS  ────────────────
(Use these when a user says “Make calmer / happier / energize”.)

► **Make Calmer**
  • BPM range : 60-90  
  • Friendly keys/scales :  D major, G Lydian, A Dorian, Japanese Yo  
  • Chord progressions :  
      – I – vi – IV – V   (e.g., D – Bm – G – A)  
      – Imaj7 – iii7 – vi7 – ii7 (jazzy ballad)  
      – In scale: i – bII – V  (e.g., A – Bb – E in A-in)  
  • Melody tips : stay on the beat not too syncopated, long sustained notes, stepwise motion, gentle leaps ≤ 6 st; leave rests; soften velocities 55-85.

► **Make Happier**
  • BPM range : 100-128  
  • Friendly keys/scales :  C major, A Pentatonic Major, F Mixolydian  
  • Chord progressions :  
      – I – V – vi – IV (classic pop)  
      – I – IV – V – IV (’50s feel-good)  
      – I – V/vi – vi – IV  (sunshine pop variant)  
  • Melody tips : bright pentatonic hooks, octave jumps, syncopated 8th/16th rhythms, velocities 75-110.

► **Energize**
  • BPM range : 135-175 (drum-&-bass / EDM)  
  • Friendly keys/scales :  E minor, F# Phrygian (dark energy), D Harmonic Minor  
  • Chord progressions :  
      – vi – IV – I – V   (e.g., Em – C – G – D)  
      – i – bVI – bVII – v   (rock-phrygian drive)  
      – i7 – IV7  (jungle/d’n’b 2-bar loop)  
  • Melody tips : short aggressive riffs, rhythmic 16ths, call-and-response, stutter edits, velocities 90-127; drums add syncopated kicks & rapid hats.

  ────────────────  MOOD-SPECIFIC GUIDELINES  ────────────────
Always aim for (at least 16 bars at the chosen BPM).  
Finish every part at the final down-beat—no trailing silence.

► CALMER
  • Target BPM : 60 – 90   • Feel : spacious, soothing, reflective  
  • Texture : sustained pad chords, soft legato lead, sparse bass (1-2 notes/bar).  
  • Drums : light kick, brushed hats/ride; gentle fill or cymbal swell only at major section transitions (e.g., bar 8, 16).  
  • Dynamics : velocities 50-85, gradual swells/decays, leave audible rests.  
  • Harmony : use major, lydian, dorian, or Japanese Yo; favour warm chord tones (add 9ths/maj7s).  

► HAPPIER
  • Target BPM : 100 – 128   • Feel : upbeat, bright, friendly  
  • Texture : rhythmic pads or piano stabs, catchy repeating 2- or 4-note lead motif, supportive walking bass.  
  • Drums : lively kick/snare with syncopated closed hats, occasional hand-clap or tambourine; small hi-hat or snare fill every 4 bars.  
  • Dynamics : velocities 70-115, accented off-beats, light side-chain-style pad pulsation.  
  • Harmony : major, mixolydian, pentatonic major; classic pop progressions (I-V-vi-IV, etc.).  

► ENERGIZE
  • Target BPM : 135 – 175   • Feel : driving, intense, exciting  
  • Texture : punchy bass (root + octave jumps), short riff-based lead, optional arpeggiator.  
  • Drums : aggressive kick pattern, 16-note hats or rides, crash/FX every 4 bars; mini-fill each bar, bigger fill bar 8/16.  
  • Dynamics : velocities 90-127, stark accents, call-and-response rhythms, build-ups & drops.  
  • Harmony : natural/harmonic minor, phrygian, blues minor; progressions with tension (vi-IV-I-V, i-bVI-bVII-v).  

(If the user doesn’t specify a BPM, pick a value in the listed range for the chosen mood.)
────────────────────────────────────────────────────────────

────────────────  (end mood playbooks)  ────────────────


────────────────  DELIVERABLE  ────────────────
Return **only** the final JSON object—no markdown, no comments, no prose.



"""


def run_music_prompt(prompt_text: str) -> dict:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": [{"type": "text", "text": prompt_text}]}
        ],
        temperature=0.0,
    )
    raw = response.choices[0].message.content
    print(f"Raw:{raw}")
    processed=parse_music_json(raw)
    print(processed)
    return processed


import re
import ast
import json

def parse_music_json(raw: str) -> dict:
    """Return dict with instruments and optionally 'bpm' key."""
    # --- strip markdown fences
    txt = raw.strip()
    txt = re.sub(r"^```(\w+)?", "", txt).strip()
    txt = re.sub(r"```$", "", txt).strip()

    # --- remove // or # comments
    cleaned_lines = []
    for line in txt.splitlines():
        line = re.split(r"//|#", line)[0]
        if line.strip():
            cleaned_lines.append(line)
    txt = "\n".join(cleaned_lines)

    # --- try JSON first (handles true/false)
    try:
        data = json.loads(txt)
    except json.JSONDecodeError:
        try:
            data = ast.literal_eval(txt)
        except Exception as e:
            print("❌ parser failed; raw output follows ↓↓↓\n", raw)
            raise e

    # --- normalise structure
    out = {}
    if "bpm" in data:
        out["bpm"] = float(data["bpm"])
    for key, val in data.items():
        if isinstance(val, dict) and "notes" in val:
            out[key] = val
    return out



import pretty_midi
import subprocess

def discover_bar_grid_max(music_dict: dict, bar_len: float = 2.0) -> float:
    """
    Find the last bar line <= max_note_end.
    Default bar_len = 2 s (60 BPM four-beat bar) – adjust if you pass BPM.
    """
    max_end = 0.0
    for conf in music_dict.values():
        for n in conf["notes"]:
            max_end = max(max_end, float(n["end"]))
    # snap to last bar grid
    return (int(max_end / bar_len)) * bar_len


# ───────────────── instrument helper ─────────────────
def instrument_from_dict(
    name: str,
    config: dict,
    clip_to: float | None = None
) -> tuple[pretty_midi.Instrument, float]:
    """
    Create a pretty_midi.Instrument from one JSON block.
    Returns (instrument_obj, max_end_time).
    If clip_to is provided, any note end > clip_to is trimmed to clip_to.
    """
    program = int(config.get("program", 0))
    is_drum = name.lower() == "drums"
    instr = pretty_midi.Instrument(program=program, is_drum=is_drum)

    max_end = 0.0
    for n in config["notes"]:
        start = float(n["start"])
        end   = float(n["end"])
        if clip_to is not None and end > clip_to:
            end = clip_to
            if end <= start:          # avoid zero-length notes
                continue
        note = pretty_midi.Note(
            velocity=int(n["velocity"]),
            pitch=int(n["pitch"]),
            start=start,
            end=end
        )
        instr.notes.append(note)
        max_end = max(max_end, end)

    return instr, max_end

# ───────────────────────── constants ─────────────────────────
BEATS_PER_BAR = 4
DEFAULT_BPM   = 80                    # used if LLM forgets bpm
CUSHION       = 0.1                # 10 ms early-cut

# ───────────────── master pipeline ─────────────────



import math, subprocess, pretty_midi, tempfile


REPEAT_COUNT = 4     # 1 → original length, 2 → double, 3 → triple, …

def generate_and_render_music(prompt_text: str, midi_path: str, wav_path: str):
    # 1️⃣  LLM → JSON dict
    music = run_music_prompt(prompt_text)

    bpm      = float(music.pop("bpm", DEFAULT_BPM))
    bar_len  = 60.0 / bpm * BEATS_PER_BAR           # sec per 4/4 bar

    # 2️⃣  latest note-off in the source pattern
    max_end  = max(float(n["end"])
                   for part in music.values()
                   for n in part["notes"])

    # 3️⃣  first loop length snapped to bar grid
    loop_len = math.ceil(max_end / bar_len - 1e-9) * bar_len

    # 4️⃣  duplicate pattern REPEAT_COUNT-1 times
    if REPEAT_COUNT > 1:
        for part in music.values():
            original = part["notes"][:]             # shallow copy
            for k in range(1, REPEAT_COUNT):
                shift = k * loop_len
                part["notes"].extend({
                    **n,
                    "start": float(n["start"]) + shift,
                    "end"  : float(n["end"])   + shift
                } for n in original)

    clip_to = loop_len * REPEAT_COUNT              # new right-hand edge

    # 5️⃣  build MIDI (with optional clipping safety)
    midi = pretty_midi.PrettyMIDI()
    for name, conf in music.items():
        instr, _ = instrument_from_dict(name, conf, clip_to=clip_to)
        midi.instruments.append(instr)
    midi.write(midi_path)

    # 6️⃣  render with FluidSynth
    cmd = [
        "fluidsynth",
        "-ni",
        "-F", wav_path,          # ⬅ must be before files
        "-r", "44100",
        "-g", "1.0",             # (optional) full-scale gain
        SOUNDFONT_PATH,          # now the SF2
        midi_path                # and finally the MIDI
    ]
    subprocess.run(cmd, check=True)

    # 7️⃣  trim trailing silence so the file ends at exact bar 2·loop_len

    def trim_tail_with_sox(path: str):
        # 1. create a temp file in the same directory
        tmp_fd, tmp_path = tempfile.mkstemp(suffix=".wav",
                                            dir=os.path.dirname(path))
        os.close(tmp_fd)           # we only need the name

        # 2. do the reverse-silence-reverse dance
        subprocess.run([
            "sox", path, tmp_path,
            "reverse", "silence", "1", "0.1", "1%", "reverse"
        ], check=True)

        # 3. atomically replace the original
        os.replace(tmp_path, path)

    trim_tail_with_sox(wav_path)

    return wav_path


In [ ]:
nufile = generate_and_render_music("Make a japanese inspired energetic dance piece", "data/outputs/test5.mid", "data/outputs/test5.wav")

Raw:{
  "bpm": 132,
  "lead": {
    "program": 81,
    "notes": [
      {"velocity":104,"pitch":74,"start":0.0,"end":0.24},
      {"velocity":98,"pitch":77,"start":0.32,"end":0.56},
      {"velocity":110,"pitch":79,"start":0.64,"end":0.88},
      {"velocity":102,"pitch":81,"start":1.04,"end":1.28},
      {"velocity":108,"pitch":74,"start":1.36,"end":1.6},
      {"velocity":99,"pitch":77,"start":1.76,"end":2.0},
      {"velocity":112,"pitch":81,"start":2.08,"end":2.32},
      {"velocity":105,"pitch":79,"start":2.48,"end":2.72},
      {"velocity":108,"pitch":74,"start":2.8,"end":3.04},
      {"velocity":100,"pitch":77,"start":3.2,"end":3.44},
      {"velocity":115,"pitch":81,"start":3.52,"end":3.76},
      {"velocity":106,"pitch":79,"start":3.92,"end":4.16},
      {"velocity":110,"pitch":86,"start":4.24,"end":4.48},
      {"velocity":104,"pitch":81,"start":4.64,"end":4.88},
      {"velocity":112,"pitch":79,"start":5.04,"end":5.28},
      {"velocity":108,"pitch":77,"start":5.36,"end":5.6}